In [1]:
import numpy as np 
import pandas as pd 
import re
import os
from plotly import tools
import plotly
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import seaborn as sns


from datetime import datetime
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('/kaggle/input/covid19-vaccine-procurement-dataset/vaccine_procurement.csv')

In [3]:
vaccine = pd.read_csv('/kaggle/input/covid-world-vaccination-progress/country_vaccinations.csv')
vaccine['date_new'] = pd.to_datetime(vaccine['date'])
covid=pd.read_csv('../input/covid19-global-dataset/worldometer_coronavirus_daily_data.csv')
covid_cum=pd.read_csv('../input/covid19-global-dataset/worldometer_coronavirus_summary_data.csv')
covid['date_new'] = pd.to_datetime(covid['date'])

In [4]:
df.head()

In [5]:
nan_vaccines=['Vector Institute_EpiVacCorona','Chumakov Centre_CoviVac','Chinese Academy of Medical Sciences_SARS-Cov-2']
df=df[~df['Company and Vaccine Name '].isin(nan_vaccines)]
df=df.replace('Sinopharm ','Sinopharm')
df=df.replace('Sinopharm(Wuhan) ','Sinopharm')
df=df.replace('Sinovac_Coronavac ','Sinovac_Coronavac')
df=df.replace('Oxford-AstraZeneca _AZD1222','Oxford-AstraZeneca')
df=df.replace('Pfizer-BioNTech_BNT162','Pfizer-BioNTech')
df=df.replace('Moderna_mRNA-1273','Moderna')
df=df.replace('Novavax_NVX-CoV2373','Novavax')
df=df.replace('Gamaleya Research Institute_Sputnik V','Sputnik_V')
df=df.replace('Janssen (J&J)_Ad26.COV2.S','Janssen(J&J)')
df=df.replace('Sanofi-GSK_SARS-CoV-2 Vaccine','Sanofi-GSK')
df=df.replace('CureVac_CVnCov','CureVac')
df=df.replace('Arcturus Therapeutics_LUNAR-COV19','Arcturus Therapeutics')
df=df.replace('Medicago_CoVLP','Medicago')
df=df.replace('COVAXX (United Biomedical)_UB-162','COVAXX')
df=df.replace('Valneva_VLA2001','Valneva')
df=df.replace('COVAX Vaccines','Covax')
df=df.replace('Bharat Biotech_COVAXIN','Bharat Biotech')
df=df.replace('CanSino Biologics_Ad5-nCoV','CanSino Biologics')
df=df.replace('Anhui Zhifei Longcom Biopharmaceutical (ZFSW)','Anhui Zhifei')

df["Purchaser Entity / Country"] = df["Purchaser Entity / Country"].str.rstrip()

df['Number of Doses Procured'] = df['Number of Doses Procured'].replace(np.nan,'0')
df['Number of Doses Procured']= df['Number of Doses Procured'].str.replace(',','')
df['Number of Doses Procured']=pd.to_numeric(df['Number of Doses Procured'], errors='coerce')

df['COVID Burden (cases/million) ']= df['COVID Burden (cases/million) '].str.replace(',','')
df['COVID Burden (cases/million) ']= df['COVID Burden (cases/million) '].str.replace(',','')
df['COVID Burden (cases/million) ']=pd.to_numeric(df['COVID Burden (cases/million) '], errors='coerce')

df[' Doses intended to be purchased  '] = df[' Doses intended to be purchased  '].replace(np.nan,'0')
df[' Doses intended to be purchased  ']= df[' Doses intended to be purchased  '].str.replace(',','')
df[' Doses intended to be purchased  ']=pd.to_numeric(df[' Doses intended to be purchased  '], errors='coerce')

df[' Number of people able to be vaccinated with doses procured ']= df[' Number of people able to be vaccinated with doses procured '].str.replace(',','')
df[' Number of people able to be vaccinated with doses procured ']= df[' Number of people able to be vaccinated with doses procured '].str.replace(',','')
df[' Number of people able to be vaccinated with doses procured ']=pd.to_numeric(df[' Number of people able to be vaccinated with doses procured '], errors='coerce')

df[' Population  '] = df[' Population  '].replace(np.nan,'0')
df[' Population  ']= df[' Population  '].str.replace(',','')
df[' Population  ']=pd.to_numeric(df[' Population  '], errors='coerce')

df['% of national population able to be vaccinated ']=df['% of national population able to be vaccinated '].str.replace('%','')
df['% of national population able to be vaccinated ']=pd.to_numeric(df['% of national population able to be vaccinated '])

df=df.replace(dict.fromkeys(['Low income ','Low income'], 'Low Income'), regex=True)
df=df.replace(dict.fromkeys(['Upper middle income ','Upper middle income'], 'Upper Middle Income'), regex=True)
df=df.replace(dict.fromkeys(['High income ','High income','High Income '], 'High Income'), regex=True)
df=df.replace(dict.fromkeys(['Lower middle income','Lower middle income ','lower middle income ','Lower Middle Income '], 'Lower Middle Income'), regex=True)
df=df.replace(dict.fromkeys(['Global Entity ','Global Entity'], 'Global Entity'), regex=True)
df=df.replace(dict.fromkeys(['Lower middle income','Lower middle income ','lower middle income ','Lower Middle Income '], 'Lower Middle Income'), regex=True)
df=df.drop(['Unnamed: 0'], axis = 1)


In [6]:
df.info()

In [7]:
df.head()

In [8]:
df.dtypes

In [9]:
df_purchase=df[['Purchaser Entity / Country','Number of Doses Procured',' Doses intended to be purchased  ']]
df_purchase=df_purchase.groupby('Purchaser Entity / Country').sum()
df_purchase = df_purchase.sort_values('Number of Doses Procured')

In [10]:
df_purchase.head()

# Exploratory Data Analysis

## Top 20 Countries who were able to procure the most vaccines

In [11]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=df_purchase.index[-20:].values,
    x=df_purchase['Number of Doses Procured'][-20:].values,
    name='Number of Doses Procured',
    orientation='h',
    marker=dict(
        color='#444224',
        
    )
))

fig.update_layout(barmode='stack',height=600,width=800,title='doses procured by top 20 countries'
    )
fig.show()

So, these are the top 20 countries who procured the highest number of vaccines by April 2020.

* USA and EU top the list as they made their first purchases for Oxford-Astrazenca vaccine in May 2020.These were quickly followed by a slate of purchases by high-income and some middle-income countries and, by the end of the summer 2020, the UK, EU countries, and Canada had purchased enough doses through advance market commitments to cover more than their entire populations. 
* The first purchases for low-income countries came in January 2021, through the African Union's pooled procurement approach.
* Many countries in Latin America, Africa, and Asia still have not yet been able to purchase enough vaccine to cover their populations.



Let's compare the number of vaccines procured to the number of vaccines intented to be procured 

In [12]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=df_purchase.index[-20:].values,
    x=df_purchase['Number of Doses Procured'][-20:].values,
    name='Number of Doses Procured',
    orientation='h',
    marker=dict(
        color='#444224',
        
    )
))
fig.add_trace(go.Bar(
    y=df_purchase.index[-20:].values,
    x=df_purchase[' Doses intended to be purchased  '][-20:].values,
    name='Doses intended to be purchased',
    orientation='h',
    marker=dict(
        color='rgba(58, 71, 80, 0.6)',
        
    )
))

fig.update_layout(barmode='stack',height=600,width=800,title='Market Commitment by top 20 countries'
    )
fig.show()

#Exploratory 

In [13]:
#numerical data
import plotly.express as px
fig = px.histogram(df_purchase, x="Number of Doses Procured")
fig.show()

In [14]:
import plotly.express as px
fig = px.box(df_purchase, y="Number of Doses Procured", points="all")
fig.show()

In [15]:
df_purchase.describe().T

In [16]:
mean=df_purchase['Number of Doses Procured'].mean();
median=df_purchase['Number of Doses Procured'].median();
mode=df_purchase['Number of Doses Procured'].mode();
fig, ax = plt.subplots(figsize=(10,6)); # — — — — — — — — -Setting size of the canvas
sns.distplot(df_purchase['Number of Doses Procured']);
plt.title('Distribution plot of Number of Doses Procured');
plt.axvline(mean,color='green',label='Mean'); 
plt.axvline(median,color='blue',label='Median');
#plt.axvline(mode[0],color='red',label='Mode1')
#plt.axvline(mode[1],color='red',label='Mode2')
plt.legend();

In [17]:
df_purchase["Number of Doses Procured"].var()

In [18]:
df_purchase["Number of Doses Procured"].std()

**Checking skewness of "Number of Doses Procured"**

In [19]:
df_purchase["Number of Doses Procured"].skew()

The distribution is positively skewed

In [20]:
df_purchase["Number of Doses Procured"].kurt()

Let's look at the relationship between actual procurement and intended procurement.

In [21]:
corr = df_purchase.corr()
sns.heatmap(corr, annot=True)

In [22]:
import plotly.express as px
fig = px.scatter(df_purchase,x=' Doses intended to be purchased  ', y='Number of Doses Procured', trendline="ols")
fig.show()

In [23]:
df_purchase.corr()

## Which vaccines were popular in major buyer countries?

Now, that we have looked at countries buying the most vaccines, let's also check the vaccines that these major buying countries have invested in.

In [24]:
from math import log, floor
def human_format(number):
    if number ==0:
        return 0
    else:
        units = ['', 'K', 'M', 'G', 'T', 'P']
        k = 1000.0
        magnitude = int(floor(log(number, k)))
        return str(int(number / k**magnitude))+ units[magnitude]

In [25]:
#categorical data
import plotly.express as px
fig = px.histogram(df, x="Company and Vaccine Name ")
fig.show()

In [26]:
import plotly.express as px
fig = px.box(df, x="Company and Vaccine Name ", points="all")
fig.show()

In [27]:
subset=['China','India','Latin America w/o Brazil','Indonesia','Brazil','UK','African Union','COVAX','USA','EU']
df_vacc=df[df['Purchaser Entity / Country'].isin(subset)]
df_vacc=df_vacc[['Company and Vaccine Name ',"Company's Country",'Purchaser Entity / Country','Number of Doses Procured']]
df_vacc = df_vacc.groupby(['Company and Vaccine Name ','Purchaser Entity / Country',"Company's Country"]).sum().reset_index()
df_vacc_cont=pd.pivot_table(df_vacc,index=['Purchaser Entity / Country','Company and Vaccine Name ',"Company's Country"])
cm = sns.light_palette("#444224", as_cmap=True)
df_vacc_cont = df_vacc_cont.style.background_gradient(cmap=cm)
# df_vacc_cont['Number of Doses Procured']=df_vacc_cont['Number of Doses Procured'].apply(human_format)
df_vacc_cont

* Among **high-income countries**, those that were home to vaccine makers were the first to secure large advance market commitments. They invested public funds into research and development of COVID-19 vaccines and leveraged their purchasing power to make large-scale deals across a portfolio of vaccine candidates.
* With limited purchasing power, **middle-income countries** used other strategies to get to the front of the queue for advance market commitments. Countries with manufacturing capacity, such as **India and Brazil**, were successful in negotiating advance market commitments with leading vaccine candidates as part of manufacturing agreements.
* Several middle-income countries, including Russia, and China, also have robust vaccine development programs and were among the first to put forward vaccine candidates and get local approvals. For countries without manufacturing or vaccine development capacity, those with the infrastructure to host clinical trials, such as **Peru**, tried to use that as leverage to negotiate purchase deals.
* **Low-income countries** that lack manufacturing and clinical testing capacity were largely left out of the deal-making process. Multilateral coalitions help address this gap, such as a pooled procurement approach led by the **African Union** resulting in hundreds of millions of doses purchased for African countries.
* However, with many manufacturing countries using export restrictions to ensure that doses meet domestic needs first, low-income countries are still at the back of the line for delivery and will likely be waiting until 2022 before they are able to cover even vulnerable populations.

## How does a country's economic status influence it's buying power?

Although it is obvious, that countries with good economic status would have procured much more vaccines than countries on the other side of the spectrum. Let's still look at the trends in vaccines purchased so far and vaccines intended to be purchased.

In [28]:
#categorical data
import plotly.express as px
fig = px.histogram(df, x="Purchaser's country Economic  Status ")
fig.show()

In [29]:
import plotly.express as px
fig = px.box(df, x="Purchaser's country Economic  Status ", points="all")
fig.show()

In [30]:
df_status=df[["Purchaser's country Economic  Status ","Number of Doses Procured",' Doses intended to be purchased  ']].groupby("Purchaser's country Economic  Status ").sum()
df_status.columns=['doses_procured','potential_dose_purchase']

In [31]:
df_status

In [32]:
corr = df.corr()
sns.heatmap(df_status, annot=True)

In [33]:
trace1=go.Bar(
x= df_status.index,
y= df_status['doses_procured'],
name='Number of Doses Procured',
marker=dict(
    color=['#a0937d','#e1701a','#f7a440','#f6dcbf','#a35709']))
trace2=go.Bar(
x= df_status.index,
y=df_status['potential_dose_purchase'],
name='Doses intended to be purchased',
marker=dict(
        color=['#a0937d','#e1701a','#f7a440','#f6dcbf','#a35709']))
fig = plotly.subplots.make_subplots(rows=1, cols=2,subplot_titles=('Confirmed Doses procured','Doses intented to be purchased'))
fig.append_trace(trace1,1,1)
fig.append_trace(trace2,1,2)

fig['layout'].update(height=600, width=800,plot_bgcolor='#faf3f3',showlegend=False,title='Market Commitment by Economic Status')
iplot(fig)

* In 2020, many **high-income countries hedged their bets by purchasing enough doses to vaccinate their populations several times over**, even before any candidates were approved and have continued to procure more doses of approved vaccines in 2021, in an effort to receive doses as quickly as possible. 
* Because of global manufacturing constraints, the direct deals made by high-income (and some middle-income) countries mean that a smaller piece of the pie is available for low- and middle-income countries and for equity-focused partnerships like COVAX in 2021.
* COVAX aims to provide around two billion doses by the end of 2021 to protect high-risk populations around the world. However, it is far from achieving this goal. Many high-income countries funded COVAX but also preferred to take on deals, so invested early in multiple vaccines, securing a place at the front of the production line.
* Forced to compete with wealthy nations for the limited number of doses coming out in 2021, **COVAX has struggled to shore up sufficient supply**. COVAX deliveries began in February 2021 but, shortly after, faxed significant supply shortages, in part stemming from India's export restrictions. 

## How will countries with different economic status be able to vaccinate their population?

In [34]:
df_pop_per=df[['Purchaser Entity / Country','Number of Doses Procured','% of national population able to be vaccinated ']]
df_pop_per=df_pop_per.groupby('Purchaser Entity / Country').sum()
df_pop_per=df_pop_per.sort_values('% of national population able to be vaccinated ')

In [35]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=df_pop_per.index[-20:].values,
    x=df_pop_per['% of national population able to be vaccinated '][-20:].values,
    name='% of national population able to be vaccinated ',
    orientation='h',
    marker=dict(
        color='#444224',
        
    )
))

fig.update_layout(barmode='stack',height=600,width=800,title='% of national population able to be vaccinated '
    )
fig.show()

In [36]:
#categorical data
import plotly.express as px
fig = px.histogram(df_pop_per, x="% of national population able to be vaccinated ")
fig.show()

In [37]:
import plotly.express as px
fig = px.box(df_pop_per, y="% of national population able to be vaccinated ", points="all")
fig.show()

In [38]:
df_pop_per.describe().T

In [39]:
mean=df_pop_per['% of national population able to be vaccinated '].mean();
median=df_pop_per['% of national population able to be vaccinated '].median();
mode=df_pop_per['% of national population able to be vaccinated '].mode();
fig, ax = plt.subplots(figsize=(10,6)); # — — — — — — — — -Setting size of the canvas
sns.distplot(df_pop_per['% of national population able to be vaccinated ']);
plt.title('Distribution plot of % of national population able to be vaccinated ');
plt.axvline(mean,color='green',label='Mean'); 
plt.axvline(median,color='blue',label='Median');
#plt.axvline(mode[0],color='red',label='Mode1')
#plt.axvline(mode[1],color='red',label='Mode2')
plt.legend();

In [40]:
df_pop_per['% of national population able to be vaccinated '].var()

In [41]:
df_pop_per['% of national population able to be vaccinated '].std()

In [42]:
df_pop_per['% of national population able to be vaccinated '].skew()

In [43]:
df_pop_per['% of national population able to be vaccinated '].kurt()

In [44]:
import plotly.express as px
fig = px.scatter(df_pop_per,x='Number of Doses Procured', y='% of national population able to be vaccinated ', trendline="ols")
fig.show()

In [45]:
corr = df_pop_per.corr()
sns.heatmap(corr, annot=True)

In [46]:
df_pop_covered=df[["Purchaser's country Economic  Status ","% of national population able to be vaccinated "]].groupby("Purchaser's country Economic  Status ").mean()

df_pop_covered.columns=['percent_population_able_to_be_vaccinated']

In [47]:
df_pop_covered.head()

In [48]:
colors=['#e1701a','#f7a440','#f6dcbf','#a35709']
fig = go.Figure(data=[go.Bar(
    x=df_pop_covered.index[1:],
    y=df_pop_covered['percent_population_able_to_be_vaccinated'][1:].values,
     marker_color=colors ,text=df_pop_covered['percent_population_able_to_be_vaccinated'][1:].values
)],layout=go.Layout(width=800,
    height=500))
fig.update_traces(texttemplate='%{text:.2s}%', textposition='outside')
fig.update_layout(title_text='% population able to be vaccinated from doses procured',plot_bgcolor='#faf3f3')

**Note that this percentage only reflects first vaccination dose and does not reflect the population that will be fully vaccinated.**
* Certainly, a country's economic status is directly proportional to the percent of the population it will be able to vaccinate. However, the gap is huge when we move from High Income Countries to Middle Income Countries, showing a need for much more equitable distribution.

Also I'm curious to know the vaccines that a country buys according to it's economic status. Let's explore that.

## Does the economic status of a country decide the vaccines they'll invest in?

In [49]:
df_vacc_status=df[['Company and Vaccine Name ',"Purchaser's country Economic  Status ",'Number of Doses Procured']].groupby(['Company and Vaccine Name ',"Purchaser's country Economic  Status "]).sum()
df_vacc_status=pd.pivot_table(df_vacc_status,index=['Company and Vaccine Name '],columns=["Purchaser's country Economic  Status "])
df_vacc_status=df_vacc_status.replace(np.nan,0)
df_vacc_status.columns=['Global Entity','High Income','Low Income','Lower Middle Income','Upper Middle Income']
df_vacc_status['total']=df_vacc_status.sum(axis=1)
df_vacc_status=df_vacc_status.sort_values('total')

In [50]:
df_vacc_status

In [51]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=df_vacc_status.index[-20:].values,
    x=df_vacc_status['Global Entity'][-20:].values,
    name='Global Entity/COVAX',
    orientation='h',
    marker=dict(
        color='#a0937d',
        
    )
))
fig.add_trace(go.Bar(
    y=df_vacc_status.index[-20:].values,
    x=df_vacc_status['High Income'][-20:].values,
    name='High Income',
    orientation='h',
    marker=dict(
        color='#e1701a',
        
    )
))
fig.add_trace(go.Bar(
    y=df_vacc_status.index[-20:].values,
    x=df_vacc_status['Low Income'][-20:].values,
    name='Low Income',
    orientation='h',
    marker=dict(
        color='#f7a440',
        
    )
))
fig.add_trace(go.Bar(
    y=df_vacc_status.index[-20:].values,
    x=df_vacc_status['Lower Middle Income'][-20:].values,
    name='Lower Middle Income',
    orientation='h',
    marker=dict(
        color='#f6dcbf',
        
    )
))
fig.add_trace(go.Bar(
    y=df_vacc_status.index[-20:].values,
    x=df_vacc_status['Upper Middle Income'][-20:].values,
    name='Upper Middle Income',
    orientation='h',
    marker=dict(
        color='#a35709',
        
    )
))

fig.update_layout(barmode='stack',plot_bgcolor='#faf3f3',height=600,width=1000,title='Vaccine Investment by Economic Status')
fig.show()

* Among high income countries, vaccines like Oxford-AstraZeneca, Pfizer-BioNTech ,Moderna, Sanofi-GSK are quite popular. 
* Several Upper Middle Income Countries like Brazil,Mexico,Argentina,Malaysia, South Africa have also invested in Oxford-AstraZeneca and Pfizer-BioNTech. Sinovac_Coronavac made in China is mostly bought by Brazil and Indonesia.
* Oxford-AstraZeneca is also the main choice of Low income countries which could be because it is now manufactured in the Serum Institue of India which would have lowered it's cost for low income countries to afford. Sputnik_V made in Russia is mostly bought by Russia and India.
* In December 2020, Janssen(J&J) [announced](https://www.jnj.com/our-company/johnson-johnson-announces-agreement-in-principle-with-gavi-to-supply-janssens-covid-19-vaccine-candidate-to-lower-income-countries-in-2021) an aggrement with COVAX to supply vaccines to lower income countries in 2021. COVAX also has two of it's own vaccines.